In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from openai import OpenAI
import os
import re
import time
import csv
from collections import Counter

In [2]:
def filter_dataframe_l(df, word_list):
    # Constructing the dynamic condition based on the list of words
    condition = df['Text'].apply(lambda x: all(re.search(re.escape(word), x, re.IGNORECASE) for word in word_list))

    # Applying the condition to filter the DataFrame
    filtered_df = df[condition]

    return filtered_df

In [3]:
def process_text(text):
    # Characters to check for
    punctuation_chars = [',', '"', "'"]

    # Iterate through each character in the text
    processed_text = ""
    for char in text:
        # Add spaces around specified punctuation characters
        if char in punctuation_chars:
            processed_text += f' \{char} '
        else:
            processed_text += char
    text = processed_text
    start_index = 0
    end_index = len(text)

    # Find the index of the first letter, <, or >
    for i, char in enumerate(text):
        if char.isalpha() or char == '<' or char == '>'or text[i] == '.':
            start_index = i
            break

    # Find the index of the last letter, <, or >
    for i in range(len(text) - 1, -1, -1):
        if text[i].isalpha() or text[i] == '<' or text[i] == '>'or text[i] == '.':
            end_index = i + 1
            break

    # Extract the substring between the first and last letters
    result = text[start_index:end_index].strip()

    result += '.'
    result = "['{}']".format(result)

    return result

In [4]:
def remove_wrapping(text):
    if isinstance(text, str) and text.startswith("['") and text.endswith("']"):
        return text[2:-2]  # Remove the first two and last two characters
    else:
        return text

In [ ]:
def remove_non_quoted_chars(text):
    #in_single_quote = False
    result = text.replace('[', '').replace(']', '')
    '''
    for char in result:
        if char == ' ' and in_single_quote:
            result += "\'"
            in_single_quote = False
        elif char == "'":
            in_single_quote = True
            result += "\'"
        else:
            result += char
    in_single_quote = False
    for char in result:
        if char == ' ' and in_single_quote:
            result += '\"'
            in_single_quote = False
        elif char == '"':
            in_single_quote = True
            result += '\"'
        else:
            result += char
    '''
    result.replace(" ' ", "@z")
    pattern = r'\'(.*?)\'|\"(.*?)\"'
    matches = re.findall(pattern, result)
    result_list = [f"'{match[0]}'" if match[0] else f'"{match[1]}"' for match in matches]
    result_f = ', '.join(result_list)
    result_f = '[' + result_f + ']'
    result_f.replace("@z", " ' ")
    return result_f

In [ ]:
def remove_non_bracket_chars(text):
    # Extract characters between brackets using a regular expression
    #text = text.replace('[\'', '').replace(']\'', '')
    #extracted_chars = re.findall(r'\[([^\]]*)\]', text)
    result = text.replace('[', '').replace(']', '')
    # Concatenate the extracted characters to form the result
    result = '[' + result + ']'
    return result

In [5]:
df = pd.read_csv('CNC_synth_copy2.csv')
df['Text'] = df['Text'].apply(lambda x: process_text(x))
df_o = pd.read_csv('CausalNewsCorpus/data/V2/train_subtask2_grouped.csv')

In [6]:
filter_list = ['<ARG0>','<ARG1>','<SIG0>','</ARG0>','</ARG1>','</SIG0>']
filter_list2 = ['<ARG0>','<ARG1>','</ARG0>','</ARG1>']

In [ ]:
df_eda = pd.read_csv('eda_2_output_transformed_train_subtask2.csv')
df_eda = df_eda[~df_eda['causal_text_w_pairs'].str.contains('\'s', case=False, na=False)]
df_eda = df_eda[~df_eda['causal_text_w_pairs'].str.contains(' \' ', case=False, na=False)]
df_eda['text'] = df_eda['text'].apply(remove_wrapping)
df_eda = replace_tokens(df_eda,'causal_text_w_pairs')
df_eda['causal_text_w_pairs'] = df_eda['causal_text_w_pairs'].apply(remove_non_bracket_chars)
df_eda['causal_text_w_pairs'] = df_eda['causal_text_w_pairs'].apply(remove_non_quoted_chars)



In [ ]:
df_t = df_eda[df_eda['num_rs'] == 1].copy()
df_t.reset_index(drop=True, inplace=True)
df_t = df_t[~df_t['causal_text_w_pairs'].str.contains('<sig1>')].copy()
df_t.reset_index(drop=True, inplace=True)
eda_result_t_df = pd.concat([df_o, df_t], ignore_index=True)
eda_result_t_df.to_csv('CNC_eda_o+orig_output55.csv', index=None)